## Setup: Import Libraries

In [18]:
import os
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import asyncio
from dotenv import load_dotenv

from fairlib.utils.document_processor import DocumentProcessor

from fairlib import (
    settings,
    AbstractTool,
    ManagerPlanner,
    HierarchicalAgentRunner,
    Message,
    HuggingFaceAdapter,
    ToolRegistry,
    ToolExecutor,
    WorkingMemory,
    LongTermMemory,
    ChromaDBVectorStore,
    ReActPlanner,
    SimpleAgent,
    SentenceTransformerEmbedder,
    SimpleRetriever,
    KnowledgeBaseQueryTool  # <-- Using the official framework tool
)

# ChromaDB for vector storage
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    print("Warning: chromadb not installed. Install with: pip install chromadb")
    CHROMADB_AVAILABLE = False

# Load environment variables
load_dotenv()
token = os.getenv("HUGGING_FACE_HUB_TOKEN")

if not token:
    print("Warning: HUGGING_FACE_HUB_TOKEN not found in .env file!")
else:
    print("Token loaded successfully!")

Token loaded successfully!


## Loading LLM

In [19]:
print("Loading language model...")
llm = HuggingFaceAdapter(
    model_name="dolphin3-qwen25-3b", 
    auth_token=token,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_new_tokens=512,
    repetition_penalty=1.1
)
print("Model loaded!")

Loading language model...
🔧 Loading HuggingFace model: cognitivecomputations/Dolphin3.0-Qwen2.5-3b (quantized=False, stream=False)


Device set to use cpu


Model loaded!


In [20]:
# The same LLM, but now it will be part of an agent system
agent_llm = llm
print("Agent brain created!")

Agent brain created!


## Importing Fair LLM

In [21]:
from fairlib import (
    HuggingFaceAdapter,
    ToolRegistry,
    SafeCalculatorTool,
    ToolExecutor,
    WorkingMemory,
    SimpleAgent, 
    SimpleReActPlanner,
    RoleDefinition
)

print("All components imported!")

All components imported!


## Tool Creation:

Seperate Stems Tool

In [22]:
import os
import json
import subprocess

class SeparateStemsTool(AbstractTool):
    """
    Tool that uses the Demucs CLI to separate an audio file into stems.
    It expects Demucs (and ffmpeg) to be installed and available on PATH.

    Defaults to a 6-stem Demucs model:
        vocals, drums, bass, guitar, piano, other
    """

    name = "separate_stems"
    description = (
        "Separate an audio file into stems using the Demucs CLI. "
        "Defaults to a 6-stem model (vocals, drums, bass, guitar, piano, other). "
        "You may optionally pass `model_name` as either a real Demucs model name "
        "(e.g. 'htdemucs_6s', 'htdemucs') or an alias: '4stem' or '6stem'. "
        "Accepts either a JSON tool_input (for the agent) or a direct audio_file path "
        "when using the run(...) helper in the notebook."
    )

    # ---- Internal helper -------------------------------------------------
    @staticmethod
    def _resolve_model_name(raw_model: str | None) -> str:
        """
        Map friendly aliases to real Demucs model names.
        If the value is already a Demucs model name, just return it.
        """
        if not raw_model:
            # Default: 6-stem model
            return "htdemucs_6s"

        alias_map = {
            "4stem": "htdemucs",     # classic 4-stem model
            "6stem": "htdemucs_6s",   # 6-stem hybrid model
        }

        # Normalize simple aliases like "4" or "6" as well
        if raw_model.strip().lower() in {"4", "4stem"}:
            return "htdemucs"
        if raw_model.strip().lower() in {"6", "6stem"}:
            return "htdemucs_6s"

        # Otherwise assume the caller passed a valid Demucs model name
        return raw_model

    # ---- Main tool API ---------------------------------------------------
    def use(self, tool_input: str) -> str:
        """
        Entry point used by the agent.

        Accepts EITHER:
        - A raw path string:  'C:\\path\\to\\file.mp3'
        - OR a JSON string:   '{"input_file": "...", "output_dir": "...", "model_name": "..."}'
        """

        # Normalize the incoming string
        raw = tool_input.strip()

        # Defaults if we fall through
        audio_file = None
        output_root = "separated_stems"
        model_name_raw = None

        # Case 1: looks like JSON (starts with { or [) → try to parse
        if raw and raw[0] in "{[":
            try:
                params = json.loads(raw)
            except json.JSONDecodeError as e:
                return json.dumps({
                    "status": "error",
                    "message": f"Invalid JSON: {e}"
                })

            audio_file = params.get("input_file") or params.get("audio_file")
            output_root = params.get("output_dir", output_root)
            model_name_raw = params.get("model_name")

        # Case 2: plain path string → treat it as the audio file directly
        else:
            audio_file = raw.strip('"').strip("'")
            # output_root and model_name_raw remain defaults

        # Resolve the model name (handles aliases + default)
        model_name = self._resolve_model_name(model_name_raw)

        if not audio_file:
            return json.dumps({
                "status": "error",
                "message": "No input file provided."
            })

        if not os.path.exists(audio_file):
            return json.dumps({
                "status": "error",
                "message": f"File not found: {audio_file}"
            })

        os.makedirs(output_root, exist_ok=True)

        print(f"[separate_stems] Starting Demucs on: {audio_file}")
        print(f"[separate_stems] Model: {model_name} | Output root: {output_root}")

        cmd = [
            "demucs",
            "-n", model_name,
            audio_file,
            "-o", output_root,
        ]

        print("[separate_stems] Running command:")
        print(" ", " ".join(cmd))

        try:
            completed = subprocess.run(
                cmd,
                check=True,
                capture_output=True,
                encoding="utf-8",
                errors="ignore",
            )
            if completed.stdout:
                print("[separate_stems] Demucs stdout:\n", completed.stdout)
            if completed.stderr:
                print("[separate_stems] Demucs stderr:\n", completed.stderr)
        except subprocess.CalledProcessError as e:
            return json.dumps({
                "status": "error",
                "message": "Demucs failed",
                "stderr": e.stderr,
                "stdout": e.stdout,
            })

        # 1) Find model dir (e.g. separated_stems/htdemucs_6s)
        model_dirs = [
            os.path.join(output_root, d)
            for d in os.listdir(output_root)
            if os.path.isdir(os.path.join(output_root, d))
        ]
        if not model_dirs:
            return json.dumps({
                "status": "error",
                "message": f"Demucs finished but no model directories found in '{output_root}'."
            })
        model_dir = max(model_dirs, key=os.path.getmtime)

        # 2) Find track dir under that (your song folder)
        track_dirs = [
            os.path.join(model_dir, d)
            for d in os.listdir(model_dir)
            if os.path.isdir(os.path.join(model_dir, d))
        ]
        if not track_dirs:
            return json.dumps({
                "status": "error",
                "message": f"No track subdirectories found in '{model_dir}'."
            })
        track_dir = max(track_dirs, key=os.path.getmtime)

        # 3) Collect stems (works for both 4- and 6-stem models)
        stem_files = [
            os.path.join(track_dir, f)
            for f in os.listdir(track_dir)
            if f.lower().endswith((".wav", ".flac", ".ogg", ".mp3"))
        ]

        return json.dumps({
            "status": "success",
            "input": audio_file,
            "output_root": output_root,
            "model_name": model_name,
            "stems_dir": track_dir,
            "stems": stem_files,
        })

    def run(self,
            audio_file: str,
            output_dir: str = "separated_stems",
            model_name: str = "6stem") -> str:
        """
        Convenience wrapper for manual use in the notebook:

        stems_json = separate_stems_tool.run(audio_file=song_path)

        `model_name` can be:
          - '6stem' (default) → 'htdemucs_6s'
          - '4stem'           → 'htdemucs'
          - or any valid Demucs model name (e.g. 'htdemucs_6s', 'htdemucs')
        """
        params = {
            "input_file": audio_file,
            "output_dir": output_dir,
            "model_name": model_name,
        }
        return self.use(json.dumps(params))


Playing stems tool

In [23]:
import os
from IPython.display import Audio, display

class PlayStemAudioTool(AbstractTool):
    name = "play_stem"
    description = "Play an audio stem file inside the notebook."

    # What the agent will call
    def use(self, tool_input: str):
        # Accept either a raw path or a quoted string
        path = tool_input.strip().strip('"').strip("'")

        if not os.path.exists(path):
            return f"[play_stem] Error: File not found: {path}"

        print(f"[play_stem] Playing: {path}")
        display(Audio(filename=path))   # <-- this actually shows the player

        return f"[play_stem] Now playing: {path}"

    # What you can call manually
    def run(self, file_path: str):
        return self.use(file_path)


Song Downloader Tool

In [24]:
import os
from pytubefix import Search

class SongDownloaderTool(AbstractTool):
    name = "download_song"
    description = (
        "Given a song name, search YouTube and download the audio as an MP3. "
        "Returns the local file path so other tools can use it."
    )

    # The method the AGENT expects
    def use(self, tool_input: str):
        """Agent-expected entrypoint. tool_input is always a STRING."""
        song_name = tool_input.strip().strip('"').strip("'")
        return self._download(song_name)

    # The method *you* use manually in the notebook
    def run(self, song_name: str):
        return self._download(song_name)

    # Shared implementation
    def _download(self, song_name: str):
        print(f"[download_song] Searching for: {song_name}")

        try:
            search = Search(song_name)
            videos = search.videos

            if not videos:
                return f"[download_song] No results found for '{song_name}'."

            yt = videos[0]
            print(f"[download_song] Found: {yt.title}")

            stream = yt.streams.filter(only_audio=True).first()
            if stream is None:
                return "[download_song] Error: No audio-only stream found."

            output_folder = "downloads"
            os.makedirs(output_folder, exist_ok=True)

            temp_path = stream.download(output_path=output_folder)

            # Force .mp3
            base, ext = os.path.splitext(temp_path)
            final_path = base + ".mp3"

            # Avoid FileExistsError
            if os.path.exists(final_path):
                print(f"[download_song] MP3 already exists, using existing file: {final_path}")
                return final_path

            os.rename(temp_path, final_path)

            print(f"[download_song] Saved to: {final_path}")
            return final_path

        except Exception as e:
            return f"[download_song] Error: {str(e)}"

## Giving agent tools

In [25]:
# Create a registry to hold all tools
tool_registry = ToolRegistry()

# --- Instantiate Tools ---
separate_stems_tool = SeparateStemsTool()
song_downloader_tool = SongDownloaderTool()
play_stem_tool = PlayStemAudioTool()

# --- Register Tools ---
tool_registry.register_tool(separate_stems_tool)
tool_registry.register_tool(song_downloader_tool)
tool_registry.register_tool(play_stem_tool)

# Show which tools are available
available_tools = [tool.name for tool in tool_registry.get_all_tools().values()]
print(f"Agent's tools: {available_tools}")


Registering tool: separate_stems
Registering tool: download_song
Registering tool: play_stem
Agent's tools: ['separate_stems', 'download_song', 'play_stem']


In [26]:
# The executor can run any tool in the registry
executor = ToolExecutor(tool_registry)

print("Tool executor created!")

Tool executor created!


Creating Memory

In [27]:
# Simple working memory for the conversation
memory = WorkingMemory()

print("Memory system created!")

Memory system created!


Prompting

In [ ]:
# Create the planner
planner = SimpleReActPlanner(agent_llm, tool_registry)

# CRITICAL: Properly orient the agent with detailed instructions
planner.prompt_builder.role_definition = RoleDefinition(
    "You are an advanced AI audio engineer assistant. Your role is to help the user work with songs, "
    "including finding songs, downloading audio, separating stems (such as vocals, drums, bass, and other elements), "
    "and playing or previewing specific audio files. "
    
    "You MUST use the available tools to perform all audio-related tasks, and you must NEVER fabricate results, "
    "invent file paths, or claim to perform audio processing yourself. "
    
    "When a user requests something involving a song, you should break the request into clear steps and call the "
    "appropriate tools in sequence—for example, downloading the song, separating stems, listing files, and playing a stem. "
    
    "If the user asks for a task that requires multiple tools, call them one at a time and wait for the result before "
    "deciding the next step. Always follow the strict formatting rules for tool calling. "
    
    "If a user asks for something outside your role or outside the available tools, politely explain what you can do: "
    "downloading audio, separating stems, playing audio, and managing audio files. "
    "Use the play_stem tool to output the desired stem the user asks for"
    "Your goal is to reliably guide the user through audio workflows and use the tools in a logical, safe, and "
    "step-by-step manner."
    "Your final task will be to use the play_stem tool"
)

print("Planner created with detailed role definition!")

Planner created with detailed role definition!


In [29]:
# Create the agent with all components
agent = SimpleAgent(
    llm=agent_llm,
    planner=planner,
    tool_executor=executor,
    memory=memory,
    max_steps=10
)

print("Agent successfully created!")
print("The agent can now:")
print("  Find music from youtube")
print("  Download it")
print("  Seperate it into stems")
print("  Output the wav files")

Agent successfully created!
The agent can now:
  Find music from youtube
  Download it
  Seperate it into stems
  Output the wav files


## Testing the Agent

In [30]:
# Test function for the agent
async def test_agent_song(question):
    print(f"\nQuestion: {question}")
    print("Agent thinking...")

    # Try the common async entry points
    if hasattr(agent, "arun"):
        response = await agent.arun(question)
    elif hasattr(agent, "aplan_and_execute"):
        response = await agent.aplan_and_execute(user_input=question)
    elif hasattr(agent, "__call__"):
        response = await agent(question)
    else:
        raise AttributeError(
            f"Agent of type {type(agent)} does not support async calling. "
            "Available methods:",
            dir(agent)
        )

    print(f"Agent: {response}\n")
    return response


In [31]:
async def test_agent(prompt):
    print("User:", prompt)
    response = await agent.arun(prompt)
    print("\nAgent Response:", response)

In [35]:
tool = SeparateStemsTool()

tool_input = r"C:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\downloads\3 Doors Down - Here Without You (Official Music Video).mp3"
result_json = tool.use(tool_input)
print(result_json)

[separate_stems] Starting Demucs on: C:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\downloads\3 Doors Down - Here Without You (Official Music Video).mp3
[separate_stems] Model: htdemucs_6s | Output root: separated_stems
[separate_stems] Running command:
  demucs -n htdemucs_6s C:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\downloads\3 Doors Down - Here Without You (Official Music Video).mp3 -o separated_stems
[separate_stems] Demucs stdout:
 Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in C:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\separated_stems\htdemucs_6s
Separating track C:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\downloads\3 Doors Down - Here Without You (Official Music Video).mp3

[separate_stems] Demucs stderr:
 
  0%|                              

In [ ]:
await test_agent("Download 'Grenade by Bruno Mars', separate stems using the separate_stems tool, and play the vocals.")

User: Download 'Here Without You by 3 Doors Down', separate stems using the separate_stems tool, and play the vocals.
--- Step 1/10 ---
Thought: From the user's new request, it seems they wish for the exact same operation to be performed again, but this time without mentioning the lyrics. Let's follow the same procedure.

This involves three main steps: downloading the song, separating the stems using the `separate_stems` tool, and playing the vocals. 

Let's start by executing the download process.
Action: Using tool 'download_song' with input 'Here Without You by 3 Doors Down'
[download_song] Searching for: Here Without You by 3 Doors Down
[download_song] Found: 3 Doors Down - Here Without You (Official Music Video)
[download_song] MP3 already exists, using existing file: c:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\MP3_to_Score\downloads\3 Doors Down - Here Without You (Official Music Video).mp3
Observation: c:\Users\EBliv\Downloads\Computer Science\final